# pySpark : Streaming Tweet

Run :
- Change credentials
- run this notebook
- run spark_streaming_03b_Tweet_spark.ipynb
- run : ~/ipynb/tweet_dashboard# python app.py

In [ ]:
#!pip requests_oauthlib

In [ ]:
import socket
import sys
import requests
import requests_oauthlib
import json

In [ ]:
# Replace the values below with yours
CONSUMER_KEY = 'P2YRxgEDOIqLPbFAlyep42Wyi'
CONSUMER_SECRET = 'hYHYbi7k78o0lHglVV5iBsXw2oAFsQl52Mjk8zj2rzLc30Tpja'
ACCESS_TOKEN = '106699008-MOW8L3gqtULlaXMZpzte9EOiVGTXrxcPjU1v5jbe'
ACCESS_SECRET = 'aZm02LUdNenTCyzlrDA9HfvMF3UoWfVlTlJu9AYJujSzE'
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)

In [ ]:
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-130,-20,100,50'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response

In [ ]:
import string
printable = set(string.printable)
def filter_str(s):
    return "".join(
        filter(
            lambda x: x in printable,
            s)
    )
# https://www.programiz.com/python-programming/methods/built-in/filter

In [ ]:
string.printable

In [ ]:
text= "Un $~étest"
"".join(filter(lambda x: x in printable, text))

In [ ]:
def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = filter_str(full_tweet['text']) + '\n'
            #print("Tweet Text: " + tweet_text)
            print("-", end='')
            #print ("------------------------------------------")
            tcp_connection.send(tweet_text.encode('utf-8'))
        except:
            e = sys.exc_info()[0]
            print("Error: %s" % e)

In [ ]:
# def send_tweets_to_spark(http_resp, tcp_connection):
#     for line in http_resp.iter_lines():
#         full_tweet = json.loads(line)
#         tweet_text = filter_str(full_tweet['text']) + '\n'
#         print(type(tweet_text))
#         print("Tweet Text: " + tweet_text)
#         print ("------------------------------------------")
#         tcp_connection.send(tweet_text.encode('utf-8'))

In [ ]:
# resp = get_tweets()
# for line in resp.iter_lines():
#     full_tweet = json.loads(line)
#     tweet_text = full_tweet['text']
#     print("Tweet Text: " + tweet_text)

In [ ]:
TCP_IP = "localhost"
TCP_PORT = 9009
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)

In [ ]:
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = get_tweets()
send_tweets_to_spark(resp, conn)

In [ ]:
s.close()